# COLLABORATIVE FILTERING
### Inspired by the research paper 
### "Time to CARE: a collaborative engine for practical disease prediction"

# Quick Introduction

#### *Collaborative Filtering is a recommender system designed to predict the preferences of a person based on the perferences of other similar users (sort of like netflix/amazon recommendations)*

#### *This technique is based on the assumption that people enjoy similar items as their peers*

#### *Applying it to healthcare data, we can generate predictions on other diseases based on a subset of similar patients*

#### *Well suited to disease prediction due to the known collaborative nature of diseases*

# Methods

<dt>1. Basic CARE framework</dt>

<dd>- Employ standard Collaborative Filtering applications</dd>

<dt>2. ICARE</dt>

<dd>- An iterative version of CARE that incorporates new elements of significance testing and ensemble methods</dd>

<dt>3. Time-sensitive ICARE</dt>

<dd>- ICARE system which uses exploits the known ordering of disease diagnoses</dd>

<dd>- These improvements make it applicable to long-term, diverse data</dd>

# Work Flow
![diagram](Diagram.png)

# Step 1: Structure the Data

In [1]:
import numpy as np
import pandas as pd
import time
import math
import sys
import csv
from datetime import datetime, date
from sklearn.preprocessing import LabelEncoder

# Some useful helper functions

In [2]:
def calculate_age(born):
    today = date.today()
    b_date = datetime.strptime(born, '%m/%d/%Y')
    return today.year - b_date.year - ((today.month, today.day) < (b_date.month, b_date.day))

def calculate_gender(gender):
    if gender == 'M':
        return 0
    elif gender == 'F':
        return 1
    else:
        return 2
    
def check_valid(code, disease_codes):

    if code == 0 or code == '-------':
        return 0

    if code in disease_codes:
        return code
    else:
        new_code = '0' + code
        if new_code in disease_codes:
            return new_code
        else:
            new_code2 = '0' + new_code
            if new_code2 in disease_codes:
                return new_code2
            else:
                return 0

def parse_diags(diag_list, disease_codes):
    new_list = []
    for diag in diag_list:
        new_diag = check_valid(diag, disease_codes)
        (new_list.append(new_diag) if new_diag is not 0 else 0)

    return new_list

# Visit class

In [3]:
class Visit:
    
    """Holds information about a visit for each patient"""

    def __init__(self, adj_date, visit):
        self.visit = visit
        self.adj_date = adj_date

    def getVisit(self):
        return self.visit

    def getDate(self):
        return self.adj_date

# Patient Class

In [4]:
class Patient:
    
    """Holds information regarding the details of each patient"""

    def __init__(self, mem_id, gender, age, visit, adj_date):
        self.visits = []
        self.mem_id = mem_id
        self.gender = gender
        self.age = age
        #v = Visit(adj_date, visit)
        self.visits.append(Visit(adj_date, visit))
        
    def getMemID(self):
        return self.mem_id

    def getGender(self):
        return self.gender

    def getAge(self):
        return self.age
    
    def getVisits(self):
        return self.visits
    
    def addVisit(self, adj_date, visit):
        self.visits.append(Visit(adj_date, visit))
        self.visits.sort(key=lambda x: x.adj_date)
        
    def getUnique(self):
        unique_codes = set()
        for visit in self.visits:
            unique_codes |= set(visit.getVisit())
        return unique_codes

# Sample line from a data file

#### TBD

# Parse CSV for dictionary

In [5]:
def parseCSV(categoryfile, labelfile):
    """
    USAGE
    categoryfile - The file provided by HCUP. Should be called '$dxref 2015.csv'
    labelfile - The file provided by HCUP. Should be called 'dxlabel 2015.csv'

    RETURNS
    dictionary - Dictionary mapping from {icd9 codes : HCUP category }
    i2d - Index to diagnosis.. { column index : ic9 diagnosis code }
    d2i - Diagnosis to index.. { ic0 diagnosis code : column index }
    
        * note * i2d and d2i includes the age and gender as the first 2 indexes
    """

    dictionary, better_dictionary, labels, d2i, i2d = {}, {}, {}, {}, {}
    diseases = []


    # add birthday & gender to the d2i and i2d dictionaries
    d2i['Age'] = 0
    i2d[0] = 'Age'
    d2i['Gender'] = 1
    i2d[1] = 'Gender'

    # parse the diagnosis code labels
    count = 0
    with open(labelfile, 'rb') as csvfile:
        datareader = csv.reader(csvfile)

        for row in datareader:
            if count > 3:
                labels[int(row[0])] = row[1]
                #maplabels[count-4] = int(row[0])
                diseases.append(row[1])

            count+=1

    csvfile.close()

    # parse the diagnosis codes file
    count = 0
    with open(categoryfile, 'rb') as csvfile:
        datareader = csv.reader(csvfile)

        for row in datareader:
            if count >= 3:
                row[0] = row[0].replace("'","").strip()
                row[1] = row[1].replace("'", "").strip()
                dictionary[row[0]] = labels[int(row[1])]
                better_dictionary[row[0]] = row[3]
                d2i[row[0]] = count - 1
                i2d[count - 1] = row[0]
            

            count+=1

    csvfile.close()

    return dictionary, i2d, d2i, diseases, better_dictionary


# Parsing CSV files, create the DataFrame

In [6]:
def cleanData(filename):
    relevant_columns = ['Member System ID', 'Adjudication Date', 'Patient Birth Date', \
                        'Patient Gender Code', 'Diagnosis One Code', \
                        'Diagnosis Two Code', 'Diagnosis Three Code', \
                        'Diagnosis Four Code', 'Diagnosis Five Code']

    df = pd.read_csv(file1, usecols=relevant_columns, dtype=np.str)\
            .drop_duplicates()\
            .reset_index().drop('index', axis=1).fillna(0)
    df = df[relevant_columns]
    df['Adjudication Date'] = pd.to_datetime(df['Adjudication Date'], format='%m/%d/%Y')
    df['Patient Birth Date'] = df['Patient Birth Date'].apply(calculate_age)
    df['Patient Gender Code'] = df['Patient Gender Code'].apply(calculate_gender)

    return df

# Create Patient Database

In [7]:
def createPatients(df, disease_codes):
    patients, diseases = {}, {}

    for row in df.itertuples():
        mem_id = row[1]
        adj_date = row[2]
        age = row[3]
        gender = row[4]
        visit = parse_diags(row[5:], disease_codes)
        
        for item in visit:
            if item not in diseases:
                diseases[item] = set()
            diseases[item].add(mem_id)

        if mem_id not in patients:
            p = Patient(mem_id, gender, age, visit, adj_date)
            patients[mem_id] = p
        else:
            patients[mem_id].addVisit(adj_date, visit)

    return patients, diseases

# Set up for CARE

In [8]:
def setupCARE(filename):
    categoryfile = '$dxref 2015.csv'
    labelfile = 'dxlabel 2015.csv'
    df = cleanData(filename)
    dic, i2d, d2i, foo_diseases, better_dic = parseCSV(categoryfile, labelfile)
    disease_codes = set(dic.keys())
    patients, diseases = createPatients(df, disease_codes)
    return patients, diseases, disease_codes, dic, better_dic

In [9]:
start_time = time.time()
file1 = 'file1.csv'
file2 = 'file2.csv'
file3 = 'file3.csv'
patients, diseases, disease_codes, dic, better_dic = setupCARE(file1)
print('--- %s seconds ---' %(time.time() - start_time))

--- 4.99566507339 seconds ---


# Step 2: Similarity Constraint for training set

#### *We obtain our training set by constraining our patient database with users with at least 2 common diseases with the target patient*

#### *This serves to remove the influence of patients with little or no similarity with the target patient*

#### *Does not result in loss of information, and reduces runtime of the algorithm*

In [10]:
def train_filter(target, patients, diseases):
    # filter patients that have at least 2 common diseases
    patient_train = {}
    disease_train = {}
    target_diseases = target.getUnique()

    #for disease in target_diseases:
    #    disease_train[disease] = set()
    #    disease_train[disease].add(target.getMemID())
    
    for patient in patients.values():
        #patient_diseases = patient.getUnique()
        combined = target_diseases & patient.getUnique()
        if len(combined) >= 2:
            patient_train[patient.getMemID()] = patient
            for disease in patient.getUnique():
                if disease not in disease_train:
                    disease_train[disease] = set()
                disease_train[disease].add(patient.getMemID())
        
    return patient_train, disease_train

# C.A.R.E

# The meat of it all:

In [11]:
def implementCARE(target, patients, diseases, disease_codes):

    ### VECTOR SIMILARITY ###
    def vote(patient, disease):
        if disease in patient.getUnique():
            return 1.0
        else:
            return 0.0

    def f(j):
        """Returns: log(# of patients in database / # of patients with disease j)"""

        return np.log( (1.0)*len(patients) / len(diseases[j]) )


    # possible optimization: turn the getUnique() set into a numpy array, then do
    # array-wise multiplication...nah
    def w(a, i):
        total_sum = 0
        combined = a.getUnique() & i.getUnique()
        for disease in combined:
            first_half = f(disease) / math.sqrt(sum(f(k)**2 for k in a.getUnique()))
            second_half = f(disease) / math.sqrt(sum(f(k)**2 for k in i.getUnique()))
            total_sum += first_half * second_half
        return total_sum

    ### PREDICTION SCORE ###
    def V(j):
        return (1.0) * len(diseases[j]) / len(patients)
    
    def V_C(j, c):
        return (1.0) * len(diseases[j] & diseases[c]) / len(patients)

    def K(a):
        return 1.0 / (sum(w(a, i) for i in patients.values()))

    def p(a, j):
        return V(j) + K(a) * (1.0 - V(j)) * (sum(w(a, patients[i]) for i in diseases[j]))

    ### BEGIN PREDICTION ###
    # TODO: need to fix & filter diseases to the train_set
    #print(p(target, '6961'))
    disease_score = []
    for disease in diseases.keys():
        score = p(target, disease)
        disease_score.append([score, disease])
    
    disease_score.sort(key = lambda x: x[0], reverse=True)

    return disease_score

# Printing functions for patients

In [12]:
def printPatient(patient, dic):
    count = 1
    print('The patient has the following diseases:')
    for disease in patient.getUnique():
        print('\t%d. ' %count + dic[disease] + ' (' + disease + ')')
        count+=1
    print('\n')

In [13]:
def printDiseases(patient, predictedDiseases, dic):
    count = 1
    print('The patient has a possibility of getting the following 10 diseases:')
    for disease in predictedDiseases:
        if disease[1] in patient.getUnique():
            continue
        print('\t%d. ' %count + dic[disease[1]] + ' (' + disease[1] + \
              ') -- ' + '{0:.2f}'.format(disease[0]))
        count+=1
        if count == 11:
            return

# Step 3: Experiments

# Create our example patient to be used as the target

In [14]:
patient_zero = Patient('1', '0', '57', ['27509', '30000', 'V700'], '05/31/1994')

printPatient(patient_zero, better_dic)

The patient has the following diseases:
	1. ROUTINE MEDICAL EXAM (V700)
	2. Disord iron metablsm NEC (Begin 2010) (27509)
	3. ANXIETY STATE NOS (30000)




# Create our training sample set from the database based on our target patient

In [15]:
patient_train, disease_train = train_filter(patient_zero, patients, diseases)

# Make CARE prediction

In [16]:
predDisease = implementCARE(patient_zero, patient_train, disease_train, disease_codes)
#printPatient(patient_zero, better_dic)
printDiseases(patient_zero, predDisease[:20], better_dic)

The patient has a possibility of getting the following 10 diseases:
	1. CIRRHOSIS OF LIVER NOS (5715) -- 0.99
	2. OTHER PSORIASIS (6961) -- 0.80
	3. LONGTERM USE OTH MED (Begin 1995) (V5869) -- 0.42
	4. ENCEPHALOPATHY- UNSPECIFIED (Begin 2003) (34830) -- 0.40
	5. Hemochromatos-rbc trans (Begin 2010) (27502) -- 0.40
	6. CLOSED SKULL VAULT FX (80000) -- 0.40
	7. SCREEN MALIG PROSTATE (Begin 1998) (V7644) -- 0.34
	8. CHR HEPAT C W/O COMA (Begin 1994) (07054) -- 0.33
	9. ELEV BL PRES W/O HYPERTN (7962) -- 0.33
	10. AC FRONTAL SINUSITIS (4611) -- 0.33


# Make ICARE Prediction